In [1]:
import pandas as pd
import os
import gene_tools as gt


os.chdir("C:/Users/alber/OneDrive/Documents/GitHub/ProjetBioinfo")

print(os.getcwd())
os.listdir("data")

C:\Users\alber\OneDrive\Documents\GitHub\ProjetBioinfo


['expl_analysis.png', '00_BPStart']

In [2]:
from gene_tools import foldersLoad, cleanDic, compute_scores, geneScores, NA_filtering,evaluate_trait_scores,evaluate_OR
from Scoring.scomet import Mean, Max, Min, Median, pca

root_path = os.path.join('data', '00_BPStart/00_gene_prioritization')

folders = ['eQTL_GWAS_blood', 'Exome', 'GWAS','pQTL_GWAS']
raw_data = foldersLoad(root_path, folders)
merged = pd.read_csv(r"data\00_BPStart\merged_drug_all_db_with_hgnc_TTD.tsv", sep="\t")
dic_clean = gt.cleanDic(raw_data)


In [3]:
from Scoring.scomet import Mean, Max, Min, Median, pca, Product

ldl_clean = dic_clean["LDL"]

Max(ldl_clean)
Mean(ldl_clean)
Median(ldl_clean)
Min(ldl_clean)
Product(ldl_clean)

score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"]
drug_targets = set(merged[merged['trait'] == 'LDL'].loc[merged[merged['trait'] == 'LDL']["Sum"] >= 3,"EnsemblId"])   
                                                                                                             
for col in score_cols:
    top = ldl_clean.loc[ldl_clean[col] < 0.5]
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")
    
for col in score_cols:
    top = ldl_clean.sort_values(by=col, ascending=True).head(int(0.01 * len(ldl_clean)))
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")


    
len(ldl_clean["GWAS_percentile"].dropna())

Prioscore_mean: 20.00% of top 1% are drug targets
Prioscore_max: 7.69% of top 1% are drug targets
Prioscore_min: 6.01% of top 1% are drug targets
Prioscore_median: 22.22% of top 1% are drug targets
Prioscore_product: 9.35% of top 1% are drug targets
Prioscore_mean: 4.41% of top 1% are drug targets
Prioscore_max: 2.94% of top 1% are drug targets
Prioscore_min: 6.37% of top 1% are drug targets
Prioscore_median: 4.90% of top 1% are drug targets
Prioscore_product: 5.39% of top 1% are drug targets


19197

In [4]:
evaluate_trait_scores(
    df = ldl_clean,
    drug_reference= merged,
    score_columns=["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median", "Prioscore_product"],
    overlap_method="topvalues",
    cutoff=0.01,
    printer=False,
    targetthreshold=3)

{'LDL': {'Prioscore_mean': 80.0,
  'Prioscore_max': 80.0,
  'Prioscore_min': 100.0,
  'Prioscore_median': 80.0,
  'Prioscore_product': 80.0}}

In [5]:


from gene_tools.analysis import evaluate_OR, run_full_trait_pipeline

evaluate_OR(merged,ldl_clean,sum_threshold=3,cutoff = 0.01,score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
            method="topvalues")


[LDL] Prioscore_mean: OR = 1.76, p = 1.3968e-01, drug targets in top 1% = 11
[LDL] Prioscore_max: OR = 1.76, p = 1.3968e-01, drug targets in top 1% = 11
[LDL] Prioscore_min: OR = 2.32, p = 3.1401e-02, drug targets in top 1% = 13
[LDL] Prioscore_median: OR = 1.76, p = 1.3968e-01, drug targets in top 1% = 11
[LDL] Prioscore_product: OR = 2.02, p = 5.8701e-02, drug targets in top 1% = 12


{'LDL': {'Prioscore_mean': [1.758, 0.14, 11],
  'Prioscore_max': [1.758, 0.14, 11],
  'Prioscore_min': [2.317, 0.031, 13],
  'Prioscore_median': [1.758, 0.14, 11],
  'Prioscore_product': [2.023, 0.059, 12]}}

In [6]:
finaldic = run_full_trait_pipeline(dic_clean,drug_reference=merged,scoring_functions = [Mean, Max, Min, Median, Product],
                                   evaluate_or_fn=evaluate_OR,evaluate_score_fn=evaluate_trait_scores,
                        score_kwargs={"cutoff": 0.005,"overlap_method": "topvalues"},or_kwargs = {"score_cols" : ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
                        "method" : "topvalues","cutoff":0.005}
                        ,verbose = False)

finaldic["TC"]


{'data':              EnsemblId Trait  eQTL_percentile  Exome_percentile  \
 0      ENSG00000130204    TC         0.007948         46.904066   
 1      ENSG00000113161    TC         0.087433          0.116928   
 2      ENSG00000189045    TC         0.031794          3.502525   
 3      ENSG00000134243    TC         0.039742         72.319957   
 4      ENSG00000109917    TC         0.294094         38.772256   
 ...                ...   ...              ...               ...   
 20428  ENSG00000224132    TC              NaN               NaN   
 20429  ENSG00000228120    TC              NaN               NaN   
 20430  ENSG00000268421    TC              NaN               NaN   
 20431  ENSG00000256566    TC              NaN               NaN   
 20432  ENSG00000177023    TC              NaN               NaN   
 
        GWAS_percentile  pQTL_percentile  Prioscore_mean  Prioscore_max  \
 0             0.206644              NaN       15.706219      46.904066   
 1             0.206644 

In [7]:
def get_best_trait_method(finaldic):
    best_trait = None
    best_method = None
    best_or = -float('inf')

    for trait, res in finaldic.items():
        for method, (or_val, p_val, n) in res['OR_eval'].items():
            if or_val > best_or:
                best_or = or_val
                best_trait = trait
                best_method = method

    return best_trait, best_method, best_or

print(f"For 0.5% : ")
finaldic = run_full_trait_pipeline(dic_clean,drug_reference=merged,scoring_functions = [Mean, Max, Min, Median, Product],
                                   evaluate_or_fn=evaluate_OR,evaluate_score_fn=evaluate_trait_scores,
                        score_kwargs={"cutoff": 0.005,"overlap_method": "topvalues"},or_kwargs = {"score_cols" : ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
                        "method" : "topvalues","cutoff":0.005}
                        ,verbose = False)
print(get_best_trait_method(finaldic))



print(f"for 1%: ")
finaldic = run_full_trait_pipeline(dic_clean,drug_reference=merged,scoring_functions = [Mean, Max, Min, Median, Product],
                                   evaluate_or_fn=evaluate_OR,evaluate_score_fn=evaluate_trait_scores,
                        score_kwargs={"cutoff": 0.01,"overlap_method": "topvalues"},or_kwargs = {"score_cols" : ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
                        "method" : "topvalues","cutoff":0.01}
                        ,verbose = False)
print(get_best_trait_method(finaldic))

print(f"for 5%: ")

finaldic = run_full_trait_pipeline(dic_clean,drug_reference=merged,scoring_functions = [Mean, Max, Min, Median, Product],
                                   evaluate_or_fn=evaluate_OR,evaluate_score_fn=evaluate_trait_scores,
                        score_kwargs={"cutoff": 0.05,"overlap_method": "topvalues"},or_kwargs = {"score_cols" : ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"],
                        "method" : "topvalues","cutoff":0.05}
                        ,verbose = False)

print(get_best_trait_method(finaldic))


For 0.5% : 
('BMD', 'Prioscore_median', inf)
for 1%: 
('EPILEPSY', 'Prioscore_mean', inf)
for 5%: 
('TC', 'Prioscore_min', 7.913)


In [8]:
from gene_tools import compare_percentiles_boxplot

ImportError: cannot import name 'compare_percentiles_boxplot' from 'gene_tools' (c:\Users\alber\OneDrive\Documents\GitHub\ProjetBioinfo\gene_tools\__init__.py)